In [28]:
import pandas as pd

In [2]:
with open('./json/australian_users_items.json','r') as read:
    data = read.readlines()

data = [eval(i) for i in data]
df = pd.DataFrame(data)

por el tamano del archivo me krashea el kernel procedo a salvarlo y trabajarlo en formato parquet
se logra ver muchos campos en la columna items en nuestro arvhivo parquet, por lo que se procede a segui aplanando datos para una mejor lectura de nuesrta data

In [54]:
# df.to_parquet('australian_users_items.parquet')
df = pd.read_parquet('./parquet/australian_users_items.parquet')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88310 entries, 0 to 88309
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           88176 non-null  object
 1   item_name         88176 non-null  object
 2   playtime_2weeks   88310 non-null  int64 
 3   playtime_forever  88310 non-null  int64 
 4   items_count       88310 non-null  int64 
 5   steam_id          88310 non-null  int64 
 6   user_url          88310 non-null  object
dtypes: int64(4), object(3)
memory usage: 5.4+ MB


es algo complejo de aplanar con tantos datos y mi equipo se ceulga, llevo 2 horas solo descomprimiendo dicha columna items

In [3]:
import ast

def safe_literal_eval(value):
  try:
      return ast.literal_eval(value)
  except (ValueError, SyntaxError):
      return value

df['items'] = df['items'].apply(safe_literal_eval)
df = df.explode('items')
df = pd.json_normalize(df['items'])

usamos funcion para comporbar campos nulos y vacios de nuestra data

In [4]:
def analizar_datos(dataframe):
    num_filas, num_columnas = dataframe.shape
   
    resultados = {}
    
    for columna in dataframe.columns:
        nulos = dataframe[columna].isnull().sum()
        vacios = dataframe[columna].apply(lambda x: x == '').sum()
        
        resultados[columna] = {
            'filas_nulas': nulos,
            'campos_vacios': vacios
        }
    
    return resultados

resultados_analisis = analizar_datos(df) 
resultados_analisis

{'item_id': {'filas_nulas': 134, 'campos_vacios': 0},
 'item_name': {'filas_nulas': 134, 'campos_vacios': 0},
 'playtime_2weeks': {'filas_nulas': 0, 'campos_vacios': 0},
 'playtime_forever': {'filas_nulas': 0, 'campos_vacios': 0},
 'items_count': {'filas_nulas': 0, 'campos_vacios': 0},
 'steam_id': {'filas_nulas': 0, 'campos_vacios': 0},
 'user_url': {'filas_nulas': 0, 'campos_vacios': 0}}

se ven muy pocos campos con datos nulos procedemos a comparar columnas y datos

In [5]:
def verificar_duplicados(df, columna1, columna2):
   return df.duplicated([columna1, columna2]).any()

In [ ]:
resultado = verificar_duplicados(df, 'user_id', 'steam_id')
resultado

In [37]:
def contar_valores_unicos(df, columna):
   
   df[columna] = df[columna].astype(str)
   numeros = df[columna].apply(lambda x: x.isdigit()).sum()
   texto = df[columna].drop_duplicates().shape[0] - numeros
   
   return numeros, texto

In [39]:
numeros, texto = contar_valores_unicos(df, 'user_id')
print(numeros)
print(texto)

56685
30941


In [51]:
numeros, texto = contar_valores_unicos(df, 'steam_id')
print(numeros)
print(texto)

88310
-685


el total de registros es de 88310; procedemos a trabajar con la columna steam_id y depurar user_id

In [67]:
df = df.drop('user_id', axis=1)

convertimos las columnas de horas jugas a tipo entero para no tener lios en las querys

In [94]:
df['playtime_2weeks'] = df['playtime_2weeks'].fillna(0).astype(int)
df['playtime_forever'] = df['playtime_forever'].fillna(0).astype(int)
df['steam_id'] = df['steam_id'].fillna(0).astype(int)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88310 entries, 0 to 88309
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           88176 non-null  object
 1   item_name         88176 non-null  object
 2   playtime_2weeks   88310 non-null  int64 
 3   playtime_forever  88310 non-null  int64 
 4   items_count       88310 non-null  int64 
 5   steam_id          88310 non-null  int64 
 6   user_url          88310 non-null  object
dtypes: int64(4), object(3)
memory usage: 5.4+ MB


In [112]:
columnas_verificar = ['steam_id', 'item_id'] 

In [113]:
duplicados = df.duplicated(subset=columnas_verificar).sum()
duplicados

1

In [20]:
df.sample(20)


,item_id,item_name,playtime_forever,items_count,steam_id,user_url
1852,202090.0,Magicka: Wizard Wars,22,155,76561198089405996,http://steamcommunity.com/id/Fluzzzer21
3390,242550.0,Rayman Legends,180,66,76561198064564908,http://steamcommunity.com/id/3265899423
19187,227940.0,Heroes & Generals,43,111,76561198075098628,http://steamcommunity.com/profiles/76561198075...
43462,352460.0,Dead Realm,68,0,76561197971923427,http://steamcommunity.com/profiles/76561197971...
42540,251170.0,Damned,125,39,76561198121991182,http://steamcommunity.com/profiles/76561198121...
22319,326960.0,Killing Floor - Toy Master,0,56,76561198070927772,http://steamcommunity.com/id/artynox
67163,212910.0,Call of Duty: Black Ops II - Zombies,1822,0,76561198071602811,http://steamcommunity.com/profiles/76561198071...
22989,333930.0,Dirty Bomb,4452,46,76561198046636820,http://steamcommunity.com/id/beanoooooo
10238,311210.0,Call of Duty: Black Ops III,908,76,76561198058103056,http://steamcommunity.com/id/YOUREALLCUNTS
53371,242050.0,Assassin's Creed IV Black Flag,2618,157,76561198048366483,http://steamcommunity.com/id/MurderTimeFunTime


la columna playtime_2weeks tiene en su mayoria campos con ceros, tomo la decision de eliminar dicha columna

In [12]:
df = df.drop('playtime_2weeks', axis=1)

In [64]:
df.sample(15)

,item_id,item_name,playtime_2weeks,playtime_forever,items_count,steam_id,user_url
25676,63710,BIT.TRIP RUNNER,0,0,249,76561198008393968,http://steamcommunity.com/profiles/76561198008...
61414,427730,Who's Your Daddy,0,176,1,76561198065406353,http://steamcommunity.com/profiles/76561198065...
82429,266840,Age of Mythology: Extended Edition,163,1204,8,76561198095263034,http://steamcommunity.com/profiles/76561198095...
83381,46000,Bob Came in Pieces,0,0,0,76561198097066185,http://steamcommunity.com/profiles/76561198097...
21932,242920,Banished,0,97,61,76561198016012144,http://steamcommunity.com/profiles/76561198016...
53612,8190,Just Cause 2,0,1936,1,76561198049044357,http://steamcommunity.com/profiles/76561198049...
33127,55110,Red Faction: Armageddon,0,168,45,76561198051361596,http://steamcommunity.com/id/montandalar
18575,386360,SMITE,269,65879,0,76561198081213404,http://steamcommunity.com/id/BlueMango101
19989,458270,CS:GO Player Profiles: Guardian - Na'Vi,0,0,0,76561198088105878,http://steamcommunity.com/id/garypoh18
49479,232210,Patch testing for Chivalry,0,72,6,76561198038095443,http://steamcommunity.com/profiles/76561198038...


In [132]:
# df.to_parquet('australian_users_items.parquet')